In [1]:
!pip install langgraph langsmith langchain langchain_groq langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.50
    Uninstalling langchain-core-0.3.

In [2]:
from langchain_groq import ChatGroq
from google.colab import userdata
groq_api_key = userdata.get('groq_api_key')

In [3]:
from typing import Dict, List, TypedDict, Annotated, Sequence, Any
from langgraph.graph import StateGraph, END
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage
from langgraph.prebuilt import ToolNode, tools_condition
import operator
from langchain_core.documents import Document
from langchain_core.runnables import RunnableConfig
import json
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
import re

from langchain_core.pydantic_v1 import BaseModel, Field
from typing_extensions import Literal
from langchain_core.prompts import ChatPromptTemplate

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
class GraphState(TypedDict):  # Class GraphState
    messages: Annotated[Sequence[BaseMessage], operator.add]
    current_agent: str
    current_llm_model: str
    context_summary: str

state = StateGraph(GraphState)

In [10]:
prompt_generator = ChatGroq(groq_api_key=groq_api_key, model="Gemma2-9b-It")
system = """You are a prompt generator for text to image model. Your task is to generate 50 diffrent prompts for text to image model to generate diverse images given to you
the scenarios. example if I give you scenario as 'people in distress in the context to disaster' you give me different prompts for image generator model to
generate a complete dataset of images of people in distress covering diverse scenarios. """
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

prompt_generator_agent = prompt | prompt_generator

def prompt_generator_handler(state: GraphState):
    """ The marketing RAG powered Agent to resolve user queries"""

    question = state["messages"][-1].content

    inbuilt_query = f""" You are a prompt generator for a text-to-image model. Given the user scenario and access to the full conversation history,
your task is to generate 50 diverse and creative image generation prompts based on the scenario provided by the user.
Make sure the prompts cover a wide range of sub-scenarios, perspectives, environments, and subjects to create a comprehensive dataset of images.
what i need as the output exactly is: return the python list of 50 prompt strings. agent_output.content should be a string like: '["prompt1", "prompt2", ...]'

User scenario:
{question}

    """
    response = prompt_generator_agent.invoke(inbuilt_query)

    # Add the question-response pair to conversation_history
    conversation_id = len(state["conversation_history"]) + 1
    state["conversation_history"][conversation_id] = {
        "question": question,
        "response": response
    }

    state["llm_model"] = "Gemma2-9b-It"

    # Also add the AI response to the messages list
    state["messages"].append(AIMessage(content=response.content))

    # Update the current_agent within the state dictionary
    state['current_agent'] = "prompt_generator_agent"  # Or the appropriate agent name

    # Return the updated state
    return state


In [6]:
chat_generator = ChatGroq(groq_api_key=groq_api_key, model="Gemma2-9b-It")
system = """You are a prompt generator for text to image model. Your task is to generate 50 diffrent prompts for text to image model to generate diverse images given to you
the scenarios. example if I give you scenario as 'people in distress in the context to disaster' you give me different prompts for image generator model to
generate a complete dataset of images of people in distress covering diverse scenarios. """
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

chat_agent = chat_prompt | chat_generator

def chat_handler(state: GraphState):
    """ The marketing RAG powered Agent to resolve user queries"""

    question = state["messages"][-1].content

    inbuilt_query = f""" You are a prompt generator for a text-to-image model. Given the user scenario and access to the full conversation history,
your task is to generate 50 diverse and creative image generation prompts based on the scenario provided by the user.
Make sure the prompts cover a wide range of sub-scenarios, perspectives, environments, and subjects to create a comprehensive dataset of images.

User scenario:
{question}
    """
    response = prompt_generator_agent.invoke(inbuilt_query)

    # Add the question-response pair to conversation_history
    conversation_id = len(state["conversation_history"]) + 1
    state["conversation_history"][conversation_id] = {
        "question": question,
        "response": response
    }

    state["llm_model"] = "Gemma2-9b-It"

    # Also add the AI response to the messages list
    state["messages"].append(AIMessage(content=response.content))

    # Update the current_agent within the state dictionary
    state['current_agent'] = "chat_agent"  # Or the appropriate agent name

    # Return the updated state
    return state


In [11]:
def main():
    # Initialize the state
    initial_state = {
        "messages": [],
        "current_agent": "prompt_generator_agent",  # Set initial agent
        "current_llm_model": "deepseek-r1-distill-qwen-32b",
        "context_summary": "",
        "conversation_history": {} # Initialize conversation history
    }

    while True:
        user_input = input("Enter a scenario (or type 'exit' to quit): ")
        if user_input.lower() == 'exit':
            break

        # Create a HumanMessage from the input
        initial_state["messages"].append(HumanMessage(content=user_input))

        # Invoke the prompt_generator_handler
        updated_state = prompt_generator_handler(initial_state)

        # Print or process the updated state's AI response
        print(updated_state["messages"][-1].content)

        # Update the initial state with the new state for the next iteration
        initial_state = updated_state

if __name__ == "__main__":
    main()


Enter a scenario (or type 'exit' to quit): Birds
```python
agent_output = [
    "A majestic bald eagle soaring above snow-capped mountains at sunset",
    "A hummingbird hovering in mid-air, sipping nectar from a vibrant flower",
    "A flock of colorful parrots squawking in a lush rainforest canopy",
    "A lone penguin waddling across a frozen Antarctic landscape",
    "A pair of lovebirds perched on a tree branch, cuddling each other",
    "A tiny wren singing a beautiful melody in a dew-kissed garden",
    "A peacock spreading its iridescent tail feathers in a display of grandeur",
    "A group of owls perched on branches, their glowing eyes observing the night",
    "A majestic owl with its wings spread wide, silhouetted against a full moon",
    "A cardinal perched on a snow-covered branch, its bright red plumage contrasting with the white",
    "A flock of starlings flying in a mesmerizing V-formation across a clear blue sky",
    "A blue jay perched on a fence post, its blue an

In [22]:
import openai
import requests
import os
import ast
from time import sleep

# Set OpenAI API key from Colab's env
# openai.api_key = os.environ["OPENAI_API_KEY"]

# Directory to save images
output_dir = "/content/generated_images"
os.makedirs(output_dir, exist_ok=True)

initial_state = {
        "messages": [],
        "current_agent": "prompt_generator_agent",  # Set initial agent
        "current_llm_model": "deepseek-r1-distill-qwen-32b",
        "context_summary": "",
        "conversation_history": {} # Initialize conversation history
    }

# === 🔁 STEP 1: Invoke your prompt generator agent ===
scenario = input("Enter a scenario (or type 'exit' to quit): ")
# agent_output = prompt_generator_agent.invoke(scenario)

initial_state["messages"].append(HumanMessage(content=scenario))

# Invoke the prompt_generator_handler
updated_state = prompt_generator_handler(initial_state)

# Print or process the updated state's AI response
agent_output = updated_state["messages"][-1].content
print(updated_state["messages"][-1].content)

# Update the initial state with the new state for the next iteration
initial_state = updated_state


# === 🧠 STEP 2: Parse output string to Python list ===
# agent_output.content should be a string like: '["prompt1", "prompt2", ...]'


try:
    # Attempt to parse as JSON first
    prompt_list = json.loads(agent_output)

    # If JSON parsing fails, fallback to ast.literal_eval
except json.JSONDecodeError:
    try:
        prompt_list = ast.literal_eval(agent_output)
    except (SyntaxError, ValueError) as e:
        # If both JSON and literal_eval fail, extract prompts using regex
        prompt_pattern = r'"(.*?)"'  # Matches text within double quotes
        prompt_list = re.findall(prompt_pattern, agent_output)
        if not prompt_list:
            raise ValueError(f"Failed to extract prompts using regex: {e}")

    if not isinstance(prompt_list, list):
        raise ValueError("Parsed output is not a list.")

except Exception as e:
    raise ValueError(f"Failed to parse prompts from agent: {e}")


print(f"✅ Got {len(prompt_list)} prompts from agent!")

# === 🎨 STEP 3: Generate and save images ===
client = openai.OpenAI(
    api_key= userdata.get("OPENAI_API_KEY")
)
for idx, prompt in enumerate(prompt_list):
    try:
        print(f"[{idx+1}/{len(prompt_list)}] Generating image for prompt: {prompt}")

        response = client.images.generate(
            model="dall-e-2",  # or "dall-e-2"
            prompt=prompt,
            size="256x256",
            quality="standard",
            n=1
        )

        image_url = response["data"][0]["url"]
        image_data = requests.get(image_url).content
        file_path = os.path.join(output_dir, f"image_{idx+1:02}.png")

        with open(file_path, 'wb') as f:
            f.write(image_data)

        print(f"✅ Saved: {file_path}")
        sleep(1)

    except Exception as e:
        print(f"❌ Error with prompt {idx+1}: {e}")


Enter a scenario (or type 'exit' to quit): trees
```python
["A towering redwood sequoia bathed in dappled sunlight, towering over a lush forest floor.", 
"A lone, gnarled oak tree standing sentinel on a windswept hilltop, branches reaching towards the stormy sky.",
"A grove of ancient olive trees, their silvery leaves shimmering in the Mediterranean sun.",
"A vibrant Japanese maple, ablaze with autumn colors, leaves cascading down like fiery rain.",
"Twisted and ancient cypress trees lining a foggy Italian path, creating an eerie and mysterious atmosphere.",
"A young sapling, its delicate leaves unfurling in the spring breeze, surrounded by wildflowers.",
"A dense forest of towering pine trees, their needles carpeting the ground in a thick layer of green.",
"A panoramic view of a sprawling forest, sunlight filtering through the leaves, creating a mosaic of light and shadow.",
"A close-up shot of a tree bark, showcasing its intricate patterns and textures.",
"A cluster of birch trees, t

KeyboardInterrupt: 